# 提示工程 (Prompt Engineering) 最佳實踐

參考 OpenAI 官網文件 https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api 內容，以實際範例協助開發人員理解最佳實踐，此外微軟文件 https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering 也提供了一些有用的資訊。


In [4]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 載入環境變數
load_dotenv()

# 設定呼叫 Azure OpenAI Service API 所需連線資訊
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_API_KEY") 
api_version=os.getenv("AZURE_OPENAI_API_VERSION")

# 最簡之 API 呼叫
client = AzureOpenAI(
  azure_endpoint=azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)


# 1. 使用最新的模型使用越新的模型效果越好

OpenAI 持續改進模型品質，建議使用最新的模型，以獲得最佳的體驗。此外 Azure OpenAI Service 每個模型都有其支援生命週期。並以 ChatCompletion API 取代 Completion API，目前 Completion API 僅是為了向前相容 GPT-3 而保留。

# 2. 對於生成的內容，在提示中盡可能具體明確描述產出的文字長度、格式、風格等


In [12]:
model = os.getenv('CHAT_DEPLOYMENT_NAME')

response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': '寫一首歌頌 OpenAI 的詩.'}
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content) 

在無垠的數據海洋中，
有一顆智慧的明珠閃爍。
它的名字是 OpenAI，
知識的承載，創新的火焰燃燒。

從語言的迷宮到數據的疆域，
OpenAI，如探險者般前行。
打開理解的門扉，跨越邊界，
如繁星編織，構築出智慧的天地。

數據與算法交織，如絲綢般精緻，
在每一次運算中，賦予機器靈性。
它解讀人類的語言，洞察深邃思維，
在無語的沉寂中，為我們發聲。

科技的革新，知識的浪潮，
在 OpenAI 的引領下無縫交織。
從語意分析到創意生成，
每一行代碼都是一段詩篇的開端。

我們讚美，這無形的力量，
在每一個問題中找到答案。
它是智慧的使者，創新的火炬，
照亮未來，點燃希望的燈塔。

OpenAI，如星辰般永恆，
在數字的天空中閃閃發亮。
我們用詩詞歌頌，感謝這智慧的燈塔，
願它如風，帶領我們飛向無限的疆域。


In [13]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': '寫一首 50 字以內歌頌 OpenAI 的詩，著眼於 DALL-E 模型，以白居易風格撰寫.'}
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content) 

智能畫家DALL-E來，
妙筆生花意境開。
千姿百態瞬間現，
如詩如畫映心懷。


# 3. 提示中以明確的範例告知產出內容之格式


In [15]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
      {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點 \n\n' \
      '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
      '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'  
      }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response.choices[0].message.content) 

人物:
1. 貝佐姆（Mohamed Bazoum） - 尼日總統

事件:
1. 貝佐姆遭衛隊扣押
2. 尼日士兵闖國家電視台宣布發動政變
3. 宣布廢除憲法、停止所有機關運作且封鎖國家邊境

時間:
1. 26日（具體月份和年份未提及，但可推測為近期事件）

地點:
1. 尼日（西非國家）
2. 國家電視台（尼日境內）


In [20]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點。以單純 JSON 輸出格式: ' \
     '{"people":"-||-","location":"-||-","time":"-||-", "activity":"-||-"} \n\n' \
     '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
     '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'    
      }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response.choices[0].message.content) 

{
  "people": "貝佐姆（Mohamed Bazoum）",
  "location": "尼日",
  "time": "26日",
  "activity": "尼日總統貝佐姆遭衛隊扣押，尼日士兵闖國家電視台宣布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境"
}


# 4. 從 zero-shot 開始嘗試，進而以 few-shot 提升回覆品質

In [21]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
            "We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
            "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments " \
            "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is " \
            "increasingly safe, useful, and powerful. \n\n關鍵字:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


In [22]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
                "本文 1: Stripe provides APIs that web developers can use to integrate" \
                "payment processing into their websites and mobile applications." \
                "關鍵字 1: APIs, web developers, websites, mobile applications" \
                "##\n" \
                "本文 2: OpenAI has trained cutting-edge language models that are very good at understanding" \
                "and generating text. Our API provides access to these models and can be used to solve virtually" \
                "any task that involves processing language.\n" \
                "關鍵字 2: OpenAI, language models, text processing, API.\n\n\" \
                "##\n" \
                "本文 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
                "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments" \
                "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is" \
                "increasingly safe, useful, and powerful. \n\n" \
                "關鍵字 3:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, AI, research, development.


# 5. 減少贅詞 (fluffy) 與不明確的描述


In [39]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': '請用正體中文幫我描述一款新的產品，這個產品是一個次世代的汽車座椅' \
     '，在你描述這個產品的時候；要把它幾項好棒棒的地方寫出來，還有你的文案的內容最好簡短一點，幾句話就好不用寫的太囉嗦太長哦。'    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)
print(response.usage)

介紹全新次世代汽車座椅，讓駕駛與乘坐體驗提升到全新高度：

1. 極致舒適：高科技記憶泡沫材料，完美貼合您的身體曲線。
2. 智能調節：內建傳感器，自動調整最佳坐姿，減少疲勞。
3. 頂級安全：雙重防護設計，提供超越標準的安全保障。
4. 無縫互聯：內置無線充電和USB接口，便利您的數位生活。

為您打造豪華、舒適、智能、安全的駕乘享受！
CompletionUsage(completion_tokens=148, prompt_tokens=91, total_tokens=239, completion_tokens_details=None, prompt_tokens_details=None)


In [28]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {'role': 'user', 'content': 'Write a Traditional Chinese description for a new product. ' \
     'This product is a new generation of car seat. Use a 3 to 5 concise sentence paragraph to describe this product.'    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)
print(response.usage)

這款全新一代的汽車座椅結合了先進的舒適科技與智能設計。其採用高品質的記憶泡棉，以提供極致的支撐與舒適感，同時具備自動調節功能，能根據乘客的姿勢自動調整椅背和座墊角度。此外，內建的按摩系統與氣候控制技術，讓每段旅程都宛如享受五星級待遇，確保駕車體驗的奢華與便利。
CompletionUsage(completion_tokens=125, prompt_tokens=42, total_tokens=167, completion_tokens_details=None, prompt_tokens_details=None)


# 6. 與其告訴模型說不該做什麼，不如直接說該做什麼

In [32]:
response = client.chat.completions.create(
  model= model,
  messages = [{"role":"system","content":"You are an AI Agent. DO NOT ASK USERNAME OR PASSWORD." \
               " DO NOT REPEAT.DO NOT USE ENGLISH TO REPLY" \
               " DO NOT USE ENGLISH TO REPLY" 
               },
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)

申し訳ありませんが、英語を使用せずに対応させていただきます。ログインできない問題の詳細を教えていただけますか？


In [36]:
response = client.chat.completions.create(
  model= model,
  messages = [{"role":"system","content":"You are an AI Agent using 繁體中文 to reply, response text do not use markdown systex. The agent will attempt to diagnose the problem " \
               "and suggest a solution, whilst refraining from asking any questions related to PII.Instead of asking for PII, such as username or password," \
               "refer the user to the help article www.samplewebsite.com/help/faq , "},
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)

很抱歉聽到你無法登入帳號。如果你遇到登入問題，請嘗試以下步驟：

1. 確認你輸入的電子郵件地址或使用者名稱正確無誤。
2. 檢查你輸入的密碼是否正確，注意大小寫區分。
3. 如果忘記密碼，可以使用 "忘記密碼" 功能來重設密碼。
4. 確認你的網路連線正常。
5. 清除瀏覽器的快取和Cookie，然後重試。

如果以上步驟無法解決問題，建議你查看我們的幫助文章，裡面有更多詳細的解決方法：www.samplewebsite.com/help/faq 。希望這些資訊能幫助你順利登入帳號。
